<a href="https://colab.research.google.com/github/Dhrumil2910/Variational-Quantum-Algorithms-for-Semidefinite-Programming-/blob/main/Algorithm_3_iVQAIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
The following code simulates iVQAIC algorithm on QASM simulator.
iVQAIC algorithm is a variational quantum algorithm for inequality constrained semidefinite programs (SDPs) that we proposed in our paper.
Here, we use iVQAIC to solve random instances of an inequality constrained SDP.
In the paper, we report simulation results for N = 8, 16, and 32, where N is the dimension of input Hermitian operators of an SDP.
For writing clean, short and understandable code, here we fix N = 8. By simply substituting another value of N and with some other minor changes, 
one can obtain results for that N.
"""

"""
This code is based on qiskit-aqua package which recently got deprecated. We intend to use new packages. However, for now the following code is based on
qiskit-aqua package. To install it, use the following pip command:

!pip install git+https://github.com/Qiskit/qiskit-aqua.git

"""

In [ ]:
# Import packages.
import cvxpy as cp
import numpy as np
import scipy as sc
import scipy.stats as stats
import scipy.sparse as sparse
import qiskit.circuit as qc
import qiskit.utils as qu
from qiskit import Aer
import qiskit.aqua as qaq
import matplotlib.pyplot as plt
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import PauliExpectation, CircuitSampler, StateFn

In [ ]:
def sparse_rand_sym(n, density):
  """
  This function generates a sparse symmetric matrix

  :param n: dimension of the matrix
  :param density: sparsity
  """
  rvs = stats.norm().rvs
  
  # generates a sparse matrix
  X = sparse.random(n, n, density=density, data_rvs=rvs)

  # make it symmetric
  upper_X = sparse.triu(X) 
  result = upper_X + upper_X.T - sparse.diags(X.diagonal())
  return result

In [ ]:
# constants of an SDP
N = 8
M = 3
R = 10

In [ ]:
# generate a weakly constrained random sparse SDP and solve them using cvxpy
C = sparse_rand_sym(N, 0.1)
A = []
b = []
for i in range(M):
    A.append(sparse_rand_sym(N, 0.1))
    b.append(np.random.randn())

# define the variable and constraints
X = cp.Variable((N,N), symmetric=True)
# positive semidefinite constraint
constraints = [X >> 0]

constraints += [
    cp.trace(A[i]@X) <= b[i] for i in range(M)
]
constraints += [
    cp.trace(X) <= R
]

# solve the problem
prob = cp.Problem(cp.Maximize(cp.trace(C@X)),
                  constraints)
prob.solve()

# result.
print("Optimal value:", prob.value)

In [ ]:
# Solving the above generated SDP with our algorithm

# define the quantum device
backend_for_simple_example = Aer.get_backend("statevector_simulator")
backend_for_simple_example_qasm = Aer.get_backend("qasm_simulator")
q_instance = QuantumInstance(backend_for_simple_example_qasm, shots=1000)

In [ ]:
# decompose the operators of the objective function and constraints into weighted Pauli strings

OperFormC = qaq.operators.MatrixOperator(C)
C_weighted_pauli = qaq.operators.op_converter.to_weighted_pauli_operator(OperFormC)

A_weighted_pauli = []
for i in range(M):
  OperFormA = qaq.operators.MatrixOperator(A[i])
  A_weighted_pauli.append(qaq.operators.op_converter.to_weighted_pauli_operator(OperFormA))

In [ ]:
learning_rate = 0.008
num_of_iterations = 54

# storing the difference between cost function value at each iteration and actual optimal value
diff_cost_func_iter_store = []

# Initialize y
y = np.zeros(M)

# iterate until convergence
# Stopping criterion can be specificed by stating number of iterations or 
# checking the norm of the full gradient of the cost function
for j in range(num_of_iterations):

  # define H(y)
  H = C_weighted_pauli.to_opflow()
  for i in range(M):
    H = H - y[i]*A_weighted_pauli[i].to_opflow()
  
  # solve inner maximization over quantum circuit parameters
  # if you switch to QASM simulator instead of Statevector simulator, change the learning step to 0.0008
  # so that it does not diverge 
  vqe = qaq.algorithms.VQE(-H)
  vqe_result = vqe.run(backend_for_simple_example)

  # evaluate the difference between current cost function value and actual optimal value
  diff_cost_func = np.dot(b, y) + R * np.max(-vqe_result.optimal_value, 0) - prob.value
  print("Step: ", j, "How far from actual optimal value: ", diff_cost_func)

  # store it
  diff_cost_func_iter_store.append(diff_cost_func)

  if vqe_result.optimal_value >= 0:
    # update the dual variable y
    for i in range(M):
      y[i] -= learning_rate*(b[i])
    continue

  # construct a quantum circuit with the optimal points obtained from the inner maximization
  optimal_circuit = vqe.construct_circuit(vqe_result['optimal_point'])

  # prepare state
  psi = qaq.operators.state_fns.CircuitStateFn(optimal_circuit[0])

  expec_A = []
  # get the expectation values for the next step
  for i in range(M):
    measurable_expression = StateFn(A_weighted_pauli[i].to_opflow(), is_measurement=True).compose(psi)
    expectation = PauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(q_instance).convert(expectation)
    expec_A.append(sampler.eval().real)

  # update the dual variable y
  for i in range(M):
    y[i] -= learning_rate*(b[i] - R * expec_A[i])

  # clean up the parameterized circuit and reset
  vqe.cleanup_parameterized_circuits()